In [1]:
import uuid
import random
import pandas as pd
from datetime import datetime
from create_sap_table import create_table_leanx as ctl

import values, helpers
import master_data, text_data, sales_doc_data

In [2]:
# master tables
for method in (
    master_data.users,
    master_data.customers_and_vendors, 
    master_data.plants, 
    master_data.materials, 
    master_data.material_support,
    master_data.routes,
    master_data.company_codes
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        df = pd.concat([all_cols, pd.DataFrame(v.values())])
        df.to_csv(f'data/OCPM/master/{table_name}.csv', index=False)

In [3]:
# text tables
for method in (
    text_data.domain_fixed_values, 
    text_data.doc_types, 
    text_data.organization,
    text_data.distribution,
    text_data.sales_doc_item_categories,
    text_data.sales_doc_rejection_reasons
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        df = pd.concat([all_cols, pd.DataFrame(v.values())])
        df.to_csv(f'data/OCPM/text/{table_name}.csv', index=False)

In [2]:
# sales order tables
sales_doc_tables = {
    'VBAK_json': {},
    'VBKD_json': {},
    'VBUK_json': {},
    'VBAP_json': {},
    'VBEP_json': {},
    'LIKP_json': {},
    'LIPS_json': {},
    'EKBE_json': {},
    'MSEG_json': {},
    'CDHDR_json': {},
    'CDPOS_json': {}
}
# create sales order
for i in range(5):
    materials=random.sample(list(values.om_materials.keys()), 5)
    quantities=[random.randint(15, 50) for _ in range(5)]
    sd = sales_doc_data.SalesAndDistribution(vbeln=str(uuid.uuid4()), materials=materials, quantities=quantities)
    sd.create_sales_order(
        sales_org=random.choice(list(values.sales_orgs.keys())),
        sales_office=None,
        distribution_channel=None,
        reqested_delivery_date=helpers.generate_random_date(start_date=datetime(2021, 1, 1), end_date=datetime(2022, 1, 1)), # TODO add custom value
        shipping_condition=random.choice(list(values.shipping_conditions.keys())),
        erdat=helpers.generate_random_date(start_date=datetime(2021, 1, 1), end_date=datetime(2022, 1, 1)),
        days_till_delivery=30,
        ernam=random.choice(list(values.om_users.keys())),
        customer=values.om_customers[random.choice(list(values.om_customers.keys()))],
    )
    sd.generate_delivery_document(
        ernam=random.choice(list(values.om_users.keys())), 
        erdat=helpers.generate_random_date(start_date=datetime(2021, 1, 1), end_date=datetime(2022, 1, 1)),
        planned_delivery_date=helpers.generate_random_date(start_date=datetime(2021, 1, 1), end_date=datetime(2022, 1, 1)),
        # picking_date=helpers.generate_random_date(start_date=datetime(2021, 1, 1), end_date=datetime(2022, 1, 1)),
        picking_date=None,
        # delivery_date=helpers.generate_random_date(start_date=datetime(2021, 1, 1), end_date=datetime(2022, 1, 1)),
        delivery_date=None,
        # confirmation_date=helpers.generate_random_date(start_date=datetime(2021, 1, 1), end_date=datetime(2022, 1, 1)),
        confirmation_date=None,
    )
    sd.pick_items(
        usnam=random.choice(list(values.om_users.keys())), 
        udate=helpers.generate_random_date(start_date=datetime(2021, 1, 1), end_date=datetime(2022, 1, 1))
    )
    sd.post_goods_issue(
        cpudt=helpers.generate_random_date(start_date=datetime(2021, 1, 1), end_date=datetime(2022, 1, 1)),
        usnam=random.choice(list(values.om_users.keys()))
    )
    sd.delivery_confirmation(
        usnam=random.choice(list(values.om_users.keys())), 
        udate=helpers.generate_random_date(start_date=datetime(2021, 1, 1), end_date=datetime(2022, 1, 1))
    )
    for k, v in sd.tables.items():
        for entry_key in list(v.keys()):
            sales_doc_tables[k][entry_key] = sd.tables[k][entry_key]

for table, rows in sales_doc_tables.items():
    table_name = table.split('_')[0]
    all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
    df = pd.concat([all_cols, pd.DataFrame(rows.values())])
    df.to_csv(f'data/OCPM/sales-document/{table_name}.csv', index=False)
